# Forecasting of Retail Sales:

## Goal

Our goal is, given data of a grocery store, predict the sales of one of their products in time. For this task we will use different machine learning models.

In [2]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv('Dataframe_Retail.csv')
df.head(15)

,Fecha,Referencia,Unidades,Ventas
0,2017-01-01,Cerveza tercio,18.000000,11.700000
1,2017-01-01,Baguetes 2 unidades,34.000000,33.320001
2,2017-01-02,Aceite de oliva virgen extra 5 l,53.000000,1184.550020
3,2017-01-02,Bolsa naranjas 2Kg.,1118.000000,1889.420064
4,2017-01-02,Mandarina,1752.468997,3356.279999
5,2017-01-02,Melo OR,399.428999,728.429999
6,2017-01-02,Baguetes 2 unidades,3572.000000,3257.839944
7,2017-01-02,Jamón reserva,48.515000,870.689999
8,2017-01-02,Cerveza tercio,1280.000000,802.150005
9,2017-01-02,Naranja en cajas,831.050001,1470.350000


## Data Analysis

What we can find in the dataframe is the sells of the each product each day and the money earned by the sale.

In [11]:
df['Referencia'].unique(), df['Referencia'].unique().shape

(array(['Cerveza tercio', 'Baguetes 2 unidades',
        'Aceite de oliva virgen extra 5 l', 'Bolsa naranjas 2Kg.',
        'Mandarina', 'Melo  OR', 'Jamón reserva', 'Naranja en cajas',
        'Pack de plátanos', 'MERLUZA PALANGRE 1 - 2 Kg (1 Caja 10Q.)',
        'Tomate jugoso al peso', 'Filetes de salmón', 'Sepia',
        'Fresa premium 500gr'], dtype=object),
 (14,))